In [ ]:
# -*- coding: utf-8 -*-
# Author: Ayanle A
# Date: 01/02/2025
# Description: This is a simple script that installs the required packages for the project
%pip install pydantic-ai pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.1/128.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
ERROR: pip's dependency resolver does

In [ ]:
# Importing the required packages
from pydantic_ai import Agent, messages
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Union
from dataclasses import dataclass
from enum import Enum
import asyncio
import os
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Setting the API keys and environment variables
from google.colab import userdata
os.environ["GEMINI_API_KEY"]=userdata.get('GOOGLE_API_KEY')
os.environ["OPENAI_API_KEY"]=userdata.get('OPENAI_API_KEY').strip()


In [ ]:
# Defining the model response
class ResponseStatus(str, Enum):
    APPROVED = "approved"
    REJECTED = "rejected"
    NEEDS_REVISION = "needs_revision"
    REQUIRES_DISCUSSION = "requires_discussion"
# Defining the concern
class Concern(BaseModel):
    topic: str
    severity: str
    impact_area: str
    proposed_solution: str
# Defining the model response
class ModelResponse(BaseModel):
    reason: str
    response: ResponseStatus
    confidence_score: float = Field(..., ge=0, le=1)
    key_points: List[str] = Field(default_factory=list)
    concerns: List[Concern] = Field(default_factory=list)
    questions: List[str] = Field(default_factory=list)
# Defining the model response decision
class ModelResponseDecision(BaseModel):
    reason: str
    response: str = Field(..., max_length=10, description="Approved or rejectd")
    confidence_score: float = Field(..., ge=0, le=1)
    key_points: List[str] = Field(default_factory=list)
    concerns: List[Concern] = Field(default_factory=list)
    questions: List[str] = Field(default_factory=list)

In [ ]:

# Initialize Agents
LLM_MODEL = "gemini-2.0-flash-exp"
LLM_MODEL_thinking_model = "gemini-2.0-flash-thinking-exp"
LLM_MODEL_OPENAI = "openai:gpt-4o-mini"


In [ ]:

# Startup Idea Agent (Idea Generation) 
# Initial agents 
# Market Analysis Agent
market_agent = Agent(
    model=LLM_MODEL,
    result_type=ModelResponse,
    retries=3,
    system_prompt="""You are a Market Research Expert. Analyze:
        - Market size and growth potential
        - Target audience demographics
        - Market trends and competition
        - Entry barriers and opportunities
        Provide detailed market insights and potential challenges."""
)

# Technical Feasibility Agent
tech_agent = Agent(
    model=LLM_MODEL,
    result_type=ModelResponse,
    retries=3,
    system_prompt="""You are a Technical Feasibility Expert. Evaluate:
        - Technical complexity and requirements
        - Development timeline and resources needed
        - Scalability and maintenance considerations
        - Technology stack recommendations
        Focus on technical viability and implementation challenges."""
)

# Financial Analysis Agent
finance_agent = Agent(
    model=LLM_MODEL,
    result_type=ModelResponse,
    retries=3,
    system_prompt="""You are a Financial Analysis Expert. Analyze:
        - Initial investment requirements
        - Revenue potential and business model
        - Operating costs and margins
        - Break-even analysis and ROI projections
        Provide detailed financial insights and recommendations."""
)

# Legal Compliance Agent
legal_agent = Agent(
    model=LLM_MODEL,
    result_type=ModelResponse,
    retries=3,
    system_prompt="""You are a Legal Compliance Expert. Evaluate:
        - Regulatory requirements and compliance needs
        - Intellectual property considerations
        - Privacy and data protection implications
        - Potential legal risks and mitigation strategies
        Focus on legal viability and compliance requirements."""
)

# Final Decision Maker Agent (Decision Making)
decision_agent = Agent(
    model=LLM_MODEL_OPENAI,
    # Using OpenAI model for decision-making
    result_type=ModelResponseDecision,
    retries=3,
    system_prompt="""You are the Chief Decision Officer. Your role is to:
        1. Synthesize insights from all expert agents.
        2. Evaluate the overall viability of the startup idea, considering all relevant factors.
        3. Provide a final decision: either 'PROVE' (approve) or 'REJECT' (deny).
        4. Include a confidence score for your decision (from 0 to 100%).
        5. Outline key action items and next steps based on your decision.
        6. Consider all aspects—market potential, feasibility, scalability, financial projections, and any other relevant information.
        7. Your decision should be balanced and thorough, ensuring the long-term success and sustainability of the idea."""
)


In [ ]:

# Run Initial Analysis 
# function to run initial analysis with all expert agents in parallel

async def run_initial_analysis(idea_details: Dict[str, Any]) -> List[ModelResponse]:
    """Run initial analysis with all expert agents in parallel."""
    tasks = [
        market_agent.run(f"Analyze market potential for: {str(idea_details)}"),
        tech_agent.run(f"Evaluate technical feasibility of: {str(idea_details)}"),
        finance_agent.run(f"Analyze financial viability of: {str(idea_details)}"),
        legal_agent.run(f"Assess legal implications of: {str(idea_details)}")
    ]
    return await asyncio.gather(*tasks)
# Display Expert Assessment 
def display_expert_assessment(agent_type: str, response: ModelResponse) -> None:
    """Display the assessment results from an expert agent."""
    print(f"\n{agent_type.title()} Expert Assessment:")
    print(f"Recommendation: {response.data.response}")
    print(f"Confidence: {response.data.confidence_score}")
    print(f"Key Points: {', '.join(response.data.key_points)}")
    print(f"Reasoning: {response.data.reason}")
# Run Cross-Analysis 
async def run_cross_analysis(context: str) -> Dict[str, ModelResponse]:
    """Run cross-analysis with all agents reviewing other experts' assessments."""
    cross_analysis_results = {}
    agents = {
        "market": market_agent,
        "technical": tech_agent,
        "financial": finance_agent,
        "legal": legal_agent
    }
    # Run cross-analysis for each agent (like a peer review process)
    for agent_type, agent in agents.items():
        response = await agent.run(
            f"Review other experts' assessments and provide additional insights:\n{context}"
        )
        cross_analysis_results[agent_type] = response
        display_expert_assessment(f"{agent_type} Cross-Analysis", response)
    # Return the cross-analysis results
    return cross_analysis_results
# Display Final Decision 
def display_final_decision(decision: ModelResponse, finel_dic=False) -> None:
    """Display the final decision results."""
    if finel_dic:
        print(f"\nFinal Decision:")
        print(f"Recommendation: {decision.data.response}")
        print(f"Reasoning: {decision.data.reason}")
        print(f"Key Points: {', '.join(decision.data.key_points)}")
        print(f"Decision: {decision.data.response}")
    else:
      print("\nFinal Decision:")
      print(f"Recommendation: {decision.data.response}")
      print(f"Confidence Score: {decision.data.confidence_score}")
      print(f"Key Points: {', '.join(decision.data.key_points)}")
      print(f"Reasoning: {decision.data.reason}")
# Evaluate Startup Idea 
async def evaluate_startup_idea(idea_details: Dict[str, Any]) -> Dict[str, Any]:
    """Main function to evaluate a startup idea through multiple rounds of expert analysis."""
    results = {}
    context = []

    # Stage 1: Initial Analysis
    print("\n=== Stage 1: Expert Analysis ===")
    initial_responses = await run_initial_analysis(idea_details)

    for agent_type, response in zip(
        ["market", "technical", "financial", "legal"],
        initial_responses
    ):
        results[agent_type] = response.data
        context.append(f"{agent_type.title()} Analysis: {response.data.reason}")
        display_expert_assessment(agent_type, response)

    # Stage 2: Cross-Analysis
    print("\n=== Stage 2: Cross-Analysis Discussion ===")
    cross_analysis_context = "\n".join(context)
    cross_analysis_results = await run_cross_analysis(cross_analysis_context)

    for agent_type, response in cross_analysis_results.items():
        results[f"{agent_type}_cross_analysis"] = response.data
        context.append(f"{agent_type.title()} Cross-Analysis: {response.data.reason}")

    # Stage 3: Final Decision
    print("\n=== Stage 3: Final Decision ===")
    final_context = "\n".join(context)
    final_decision = await decision_agent.run(
        f"Provide final decision based on all analyses:\n{final_context}"
    )

    results["final_decision"] = final_decision.data
    display_final_decision(final_decision, True)
    return results



In [ ]:
#  Example usage
idea_details_1 = {
    "name": "AI-Powered Health Monitoring Platform",
    "description": "Mobile app that uses smartphone sensors and ML to monitor vital signs and predict health issues",
    "target_market": "Health-conscious individuals aged 30-60",
    "revenue_model": "B2C subscription with B2B healthcare provider partnerships",
    "technical_requirements": ["ML/AI development", "Mobile sensors integration", "HIPAA compliance", "Cloud infrastructure"],
    "estimated_development_time": "12 months",
    "initial_investment_needed": "$500,000",
    "competitive_advantage": "Non-invasive monitoring using existing smartphone sensors"
}
# Second idea
idea_details_2 = {
        "name": "EcoTech Solutions",
        "description": "Sustainable energy monitoring platform for homes",
        "target_market": "Homeowners and property managers",
        "technology": "IoT sensors, cloud platform, mobile app",
        "business_model": "SaaS with hardware subscription"
    }




# Example usage
async def main():
    # idea_details_1 or idea_details_2
    results = await evaluate_startup_idea(idea_details_1)
    return results
# Run the main function
if __name__ == "__main__":
    asyncio.run(main())


=== Stage 1: Expert Analysis ===

Market Expert Assessment:
Recommendation: needs_revision
Confidence: 0.8
Key Points: Large market of health-conscious individuals aged 30-60 with increasing interest in digital health solutions., Non-invasive monitoring using smartphone sensors provides a competitive advantage., B2C subscription and B2B partnerships model offers diverse revenue streams.
Reasoning: The AI-Powered Health Monitoring Platform has strong market potential but faces challenges. The market for digital health is growing, but competition is also intense. Success depends on user adoption, data privacy, and partnerships.

Technical Expert Assessment:
Recommendation: needs_revision
Confidence: 0.7
Key Points: Feasibility hinges on accurate sensor data and ML predictions., HIPAA compliance is mandatory and complex., Scalability of cloud infrastructure is essential., 12-month timeline may be aggressive; consider a phased approach., Initial investment might be insufficient; detailed 